# 翻頁

In [4]:
# 導入庫
import os
import time
import pymysql
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.parse import quote
from datetime import datetime

# 載入環境變數
load_dotenv()
EMAIL = os.getenv("COUPANG_EMAIL")
PASSWORD = os.getenv("COUPANG_PASSWORD")

DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT", 3306)),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME")
}

# 登入函數
def login_and_get_driver(email, password):
    options = Options()
    # options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--lang=zh-TW")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    driver.get("https://member.tw.coupang.com/login/login.pang")
    time.sleep(2)

    driver.find_element(By.ID, "login-email-input").send_keys(email)
    driver.find_element(By.ID, "login-password-input").send_keys(password)
    driver.find_element(By.CLASS_NAME, "login__button--submit").click()
    time.sleep(5)
    print("✅ 登入成功")
    return driver

# 爬蟲函數（含翻頁）
def get_search_results(driver, keyword, max_pages=5):
    encoded = quote(keyword)
    now = datetime.now()
    results = []

    for page in range(1, max_pages + 1):
        url = f"https://www.tw.coupang.com/search?q={encoded}&channel=user&page={page}"
        driver.get(url)
        time.sleep(5)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = soup.select("div.SearchResult_searchResultProduct___h6E9")

        if not cards:
            print(f"⚠️ 第 {page} 頁無資料，停止翻頁。")
            break

        for card in cards:
            try:
                full_text = card.get_text(" ", strip=True)
                title = card.select_one("div.Product_title__8K0xk")
                price = card.select_one("span.Product_salePricePrice__2FbsL span")
                unit_price = card.select_one("div.Product_unitPrice__QQPdR")
                results.append({
                    "search_keyword": keyword,
                    "title": title.get_text(strip=True) if title else "N/A",
                    "full_text": full_text,
                    "price": price.get_text(strip=True) if price else "N/A",
                    "unit_price": unit_price.get_text(strip=True) if unit_price else "N/A",
                    "timestamp": now
                })
            except Exception as e:
                print("❌ 錯誤：", e)
                continue
        print(f"✅ 第 {page} 頁擷取完成，共 {len(cards)} 筆")
    return results

# 寫入資料庫
def insert_into_db(data_list, db_config):
    conn = pymysql.connect(
        host=db_config["host"],
        port=db_config["port"],
        user=db_config["user"],
        password=db_config["password"],
        database=db_config["database"],
        charset="utf8mb4"
    )
    with conn:
        with conn.cursor() as cursor:
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS coupang_products (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    search_keyword VARCHAR(255),
                    title TEXT,
                    full_text TEXT,
                    price VARCHAR(50),
                    unit_price VARCHAR(50),
                    timestamp DATETIME
                );
            """)
            for item in data_list:
                cursor.execute("""
                    INSERT INTO coupang_products (search_keyword, title, full_text, price, unit_price, timestamp)
                    VALUES (%s, %s, %s, %s, %s, %s);
                """, (
                    item["search_keyword"], item["title"], item["full_text"],
                    item["price"], item["unit_price"], item["timestamp"]
                ))
        conn.commit()
    print("✅ 已寫入資料庫")

# 主程式
if __name__ == "__main__":
    keyword = "BLUE BAY 倍力 Sense"
    driver = login_and_get_driver(EMAIL, PASSWORD)
    results = get_search_results(driver, keyword, max_pages=5)
    driver.quit()

    if results:
        for idx, r in enumerate(results, 1):
            print(f"{idx}. {r['title']} - {r['price']} - {r['unit_price']}")
        insert_into_db(results, DB_CONFIG)
    else:
        print("⚠️ 查無結果")

✅ 登入成功
✅ 第 1 頁擷取完成，共 30 筆
✅ 第 2 頁擷取完成，共 30 筆
✅ 第 3 頁擷取完成，共 30 筆
✅ 第 4 頁擷取完成，共 30 筆
✅ 第 5 頁擷取完成，共 30 筆
1. BLUE BAY 倍力 Sense 全齡貓 全護低敏 貓飼料 田野雞, 腸胃保健, 1.5kg, 1袋 - $304 - ($20.27/100g)
2. BLUE BAY 倍力 Sense 全護低敏貓飼料 美膚爆毛配方 3種魚 + 台灣鱉蛋, 皮膚/毛髮, 1.5kg, 1袋 - $341 - ($22.73/100g)
3. BLUE BAY 倍力 Sense 各階段成幼貓 全護低敏貓飼料 雞肉 + 鮭魚, 高纖化毛, 13.6kg, 1袋 - $2,042 - ($15.02/100g)
4. BLUE BAY 倍力 S30 低敏狗糧 雞肉 + 燕麥, 心血管保健, 15kg, 1袋 - $1,984 - ($13.23/100g)
5. BLUE BAY 倍力 Sense 全護低敏貓飼料 美膚爆毛配方 3種魚 + 台灣鱉蛋, 皮膚/毛髮, 1.5kg, 3袋 - $1,087 - ($24.16/100g)
6. BLUE BAY 倍力 Sense 全護低敏貓飼料 美膚爆毛配方 3種魚 + 台灣鱉蛋, 皮膚/毛髮, 1.5kg, 2袋 - $708 - ($23.60/100g)
7. BLUE BAY 倍力 S30 舒敏護膚配方, 舒敏護膚, 7.5kg, 1袋 - $1,008 - ($13.44/100g)
8. BLUE BAY 倍力 Sense 全護低敏貓飼料 美膚爆毛配方 3種魚 + 台灣鱉蛋, 皮膚/毛髮, 1.5kg, 4袋 - $1,466 - ($24.43/100g)
9. BLUE BAY 倍力 成犬S30關節保健低敏配方狗飼料 羊肉 + 南瓜, 關節保健, 7.5kg, 1袋 - $1,008 - ($13.44/100g)
10. BLUE BAY 倍力 S30舒敏護膚配方 狗飼料, 鮭魚 + 甜薯, 1.5kg, 1袋 - $304 - ($20.27/100g)
11. BLUE BAY 倍力 舒敏護膚配方 犬用乾飼料, 鮭魚 + 甜薯, 16kg, 1袋 - $1,995 - ($12.47/100g)
12. BLU

# 添加類型、寫入資料庫

In [7]:
import os
import time
import pymysql
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.parse import quote
from datetime import datetime

# 載入環境變數
load_dotenv()
EMAIL = os.getenv("COUPANG_EMAIL")
PASSWORD = os.getenv("COUPANG_PASSWORD")

DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT", 3306)),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME")
}

# 登入並取得 driver
def login_and_get_driver(email, password):
    options = Options()
    # options.add_argument("--headless=new")  # 若需無頭模式可取消註解
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--lang=zh-TW")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )

    driver.get("https://member.tw.coupang.com/login/login.pang")
    time.sleep(2)

    driver.find_element(By.ID, "login-email-input").send_keys(email)
    driver.find_element(By.ID, "login-password-input").send_keys(password)
    driver.find_element(By.CLASS_NAME, "login__button--submit").click()
    time.sleep(5)
    print("✅ 登入成功")
    return driver

# 爬蟲含翻頁與來源欄位
def get_search_results(driver, keyword, max_pages=5):
    encoded = quote(keyword)
    now = datetime.now()
    results = []

    for page in range(1, max_pages + 1):
        url = f"https://www.tw.coupang.com/search?q={encoded}&channel=user&page={page}"
        driver.get(url)
        time.sleep(5)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = soup.select("div.SearchResult_searchResultProduct___h6E9")

        if not cards:
            print(f"⚠️ 第 {page} 頁無資料，停止翻頁。")
            break

        for card in cards:
            try:
                full_text = card.get_text(" ", strip=True)
                title = card.select_one("div.Product_title__8K0xk")
                price = card.select_one("span.Product_salePricePrice__2FbsL span")
                unit_price = card.select_one("div.Product_unitPrice__QQPdR")

                results.append({
                    "search_keyword": keyword,
                    "title": title.get_text(strip=True) if title else "N/A",
                    "full_text": full_text,
                    "price": price.get_text(strip=True) if price else "N/A",
                    "unit_price": unit_price.get_text(strip=True) if unit_price else "N/A",
                    "timestamp": now,
                    "source_type": "login"
                })
            except Exception as e:
                print("❌ 錯誤：", e)
                continue
        print(f"✅ 第 {page} 頁擷取完成，共 {len(cards)} 筆")
    return results

# 寫入資料庫（自動檢查 source_type 欄位）
def insert_into_db(data_list, db_config):
    conn = pymysql.connect(
        host=db_config["host"],
        port=db_config["port"],
        user=db_config["user"],
        password=db_config["password"],
        database=db_config["database"],
        charset="utf8mb4"
    )

    with conn:
        with conn.cursor() as cursor:
            # 檢查欄位 source_type 是否存在，若無則新增
            cursor.execute("SHOW COLUMNS FROM coupang_products LIKE 'source_type';")
            if not cursor.fetchone():
                cursor.execute("ALTER TABLE coupang_products ADD COLUMN source_type VARCHAR(20) NULL;")

            cursor.execute("""
                CREATE TABLE IF NOT EXISTS coupang_products (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    search_keyword VARCHAR(255),
                    title TEXT,
                    full_text TEXT,
                    price VARCHAR(50),
                    unit_price VARCHAR(50),
                    timestamp DATETIME,
                    source_type VARCHAR(20) NULL
                );
            """)

            for item in data_list:
                cursor.execute("""
                    INSERT INTO coupang_products
                    (search_keyword, title, full_text, price, unit_price, timestamp, source_type)
                    VALUES (%s, %s, %s, %s, %s, %s, %s);
                """, (
                    item["search_keyword"],
                    item["title"],
                    item["full_text"],
                    item["price"],
                    item["unit_price"],
                    item["timestamp"],
                    item.get("source_type")
                ))
        conn.commit()
    print("✅ 已寫入資料庫")

# 主程式
if __name__ == "__main__":
    keyword = "BLUE BAY 倍力 Animate"
    driver = login_and_get_driver(EMAIL, PASSWORD)
    results = get_search_results(driver, keyword, max_pages=5)
    # driver.quit()

    if results:
        for idx, r in enumerate(results, 1):
            print(f"{idx}. {r['title']} - {r['price']} - {r['unit_price']} - 來源: {r.get('source_type')}")
        insert_into_db(results, DB_CONFIG)
    else:
        print("⚠️ 查無結果")


✅ 登入成功
✅ 第 1 頁擷取完成，共 30 筆
✅ 第 2 頁擷取完成，共 30 筆
✅ 第 3 頁擷取完成，共 30 筆
✅ 第 4 頁擷取完成，共 30 筆
✅ 第 5 頁擷取完成，共 30 筆
1. BLUE BAY 倍力 挑嘴全齡貓 Animate 免疫防護配方 無穀天然鮮貓糧, 極上海鮮, 1.5kg, 1袋 - $261 - ($17.40/100g) - 來源: login
2. BLUE BAY 倍力 Animate 室內低磷配方 乾飼料, 海魚 + 鮮雞, 4.5kg, 1袋 - $795 - ($17.67/100g) - 來源: login
3. BLUE BAY 倍力 挑嘴全齡貓 Animate 免疫防護配方 無穀天然鮮貓糧, 極上海鮮, 4.5kg, 1袋 - $1,111 - ($24.69/100g) - 來源: login
4. BLUE BAY 倍力 Sense 全齡貓 全護低敏 貓飼料 田野雞, 腸胃保健, 1.5kg, 1袋 - $304 - ($20.27/100g) - 來源: login
5. BLUE BAY 倍力 Animate室內低磷配方乾飼料, 海魚 + 鮮雞, 1.5kg, 1袋 - $338 - ($22.53/100g) - 來源: login
6. BLUE BAY 倍力 Animate室內低磷配方乾飼料, 海魚 + 鮮雞, 1.5kg, 2袋 - $550 - ($18.33/100g) - 來源: login
7. BLUE BAY 倍力 Animate 室內低磷配方 乾飼料, 海魚 + 鮮雞, 4.5kg, 2袋 - $1,530 - ($17.00/100g) - 來源: login
8. BLUE BAY 倍力 海島貓咪主食罐 鬼頭刀燉牛, 壓力緩解/安撫情緒/注意力集中, 80g, 3罐 - $165 - ($68.75/100g) - 來源: login
9. BLUE BAY 倍力 S30 舒敏護膚配方, 舒敏護膚, 7.5kg, 1袋 - $1,008 - ($13.44/100g) - 來源: login
10. BLUE BAY 倍力 成犬S30關節保健低敏配方狗飼料 羊肉 + 南瓜, 關節保健, 7.5kg, 1袋 - $1,008 - ($13.44/100g) - 來源: l

# 僅顯示 Login

其餘代碼與前面相同

In [8]:
import pymysql
import os
from dotenv import load_dotenv
from collections import defaultdict

# 載入設定
load_dotenv()
db_config = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT", 3306)),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME")
}

# 查詢條件
search_keyword = "BLUE BAY 倍力"
# 進階篩選條件（title 中必須同時包含所有這些關鍵字）
filter_keywords_in_title = ["室內低磷"]

# SQL 查詢（包含登入來源欄位）
# sql = """
#     SELECT title, price, timestamp, source_type
#     FROM coupang_products
#     WHERE search_keyword LIKE %s
#     ORDER BY title, timestamp
# """
# SQL 查詢（僅抓取 login 資料）
sql = """
    SELECT title, price, timestamp, source_type
    FROM coupang_products
    WHERE search_keyword LIKE %s AND source_type = 'login'
    ORDER BY title, timestamp
"""


# 建立連線並查詢
connection = pymysql.connect(
    host=db_config["host"],
    port=db_config["port"],
    user=db_config["user"],
    password=db_config["password"],
    database=db_config["database"],
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with connection:
    with connection.cursor() as cursor:
        cursor.execute(sql, (f"%{search_keyword}%",))
        rows = cursor.fetchall()

# 進階條件篩選（title 同時包含所有關鍵字）
def is_match_advanced(title: str, keywords: list[str]) -> bool:
    return all(kw in title for kw in keywords)

if filter_keywords_in_title:
    rows = [r for r in rows if is_match_advanced(r['title'], filter_keywords_in_title)]

# 分組：依 title 對應多筆 (價格, 時間, 來源)
grouped = defaultdict(list)
for row in rows:
    grouped[row['title']].append((row['price'], row['timestamp'], row['source_type'] or "guest"))

# 僅保留有兩種以上價格的 title（根據不同價格去重）
filtered_grouped = {
    title: entries
    for title, entries in grouped.items()
    if len(set(price for price, _, _ in entries)) >= 2
}

# 輸出結果
print(f"📌 搜尋條件：search_keyword 含『{search_keyword}』")
if filter_keywords_in_title:
    print(f"🎯 進階條件：title 同時包含『{'、'.join(filter_keywords_in_title)}』\n")

if not filtered_grouped:
    print("⚠️ 查無重複價格變化的項目")
else:
    for title, records in filtered_grouped.items():
        print(f"🔹 標題：{title}")
        for price, timestamp, source in sorted(records, key=lambda x: x[1]):
            print(f"   - 價格：{price}（時間：{timestamp}，來源：{source}）")
        print("-" * 60)

📌 搜尋條件：search_keyword 含『BLUE BAY 倍力』
🎯 進階條件：title 同時包含『室內低磷』

🔹 標題：BLUE BAY 倍力 Animate 室內低磷配方 乾飼料, 海魚 + 鮮雞, 4.5kg, 1袋
   - 價格：$791（時間：2025-05-18 02:51:09，來源：login）
   - 價格：$791（時間：2025-05-18 15:34:38，來源：login）
   - 價格：$795（時間：2025-05-19 03:27:47，來源：login）
   - 價格：$795（時間：2025-05-19 04:08:37，來源：login）
   - 價格：$795（時間：2025-05-19 04:08:37，來源：login）
   - 價格：$795（時間：2025-05-19 04:08:37，來源：login）
   - 價格：$795（時間：2025-05-19 04:08:37，來源：login）
   - 價格：$795（時間：2025-05-19 04:08:37，來源：login）
   - 價格：$795（時間：2025-05-19 04:10:15，來源：login）
   - 價格：$795（時間：2025-05-19 04:10:15，來源：login）
   - 價格：$795（時間：2025-05-19 04:10:15，來源：login）
   - 價格：$795（時間：2025-05-19 04:10:15，來源：login）
   - 價格：$795（時間：2025-05-19 04:10:15，來源：login）
------------------------------------------------------------


# 輸出時去除重複

In [9]:
import pymysql
import os
from dotenv import load_dotenv
from collections import defaultdict

# 載入設定
load_dotenv()
db_config = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT", 3306)),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME")
}

# 查詢條件
search_keyword = "BLUE BAY 倍力"
# 進階篩選條件（title 中必須同時包含所有這些關鍵字）
filter_keywords_in_title = ["室內低磷"]

# SQL 查詢（包含登入來源欄位）
# sql = """
#     SELECT title, price, timestamp, source_type
#     FROM coupang_products
#     WHERE search_keyword LIKE %s
#     ORDER BY title, timestamp
# """
# SQL 查詢（僅抓取 login 資料）
sql = """
    SELECT title, price, timestamp, source_type
    FROM coupang_products
    WHERE search_keyword LIKE %s AND source_type = 'login'
    ORDER BY title, timestamp
"""


# 建立連線並查詢
connection = pymysql.connect(
    host=db_config["host"],
    port=db_config["port"],
    user=db_config["user"],
    password=db_config["password"],
    database=db_config["database"],
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with connection:
    with connection.cursor() as cursor:
        cursor.execute(sql, (f"%{search_keyword}%",))
        rows = cursor.fetchall()

# 進階條件篩選（title 同時包含所有關鍵字）
def is_match_advanced(title: str, keywords: list[str]) -> bool:
    return all(kw in title for kw in keywords)

if filter_keywords_in_title:
    rows = [r for r in rows if is_match_advanced(r['title'], filter_keywords_in_title)]

# 分組：依 title 對應多筆 (價格, 時間, 來源)
grouped = defaultdict(list)
# 記錄 (title, timestamp)
seen = set()
# 去重複
for row in rows:
    key = (row['title'], row['timestamp'])
    if key in seen:
        # 跳過完全重複的記錄
        continue
    # 標記為已見
    seen.add(key)

    grouped[row['title']].append((row['price'], row['timestamp'], row['source_type'] or "guest"))

# 僅保留有兩種以上價格的 title（根據不同價格去重）
filtered_grouped = {
    title: entries
    for title, entries in grouped.items()
    if len(set(price for price, _, _ in entries)) >= 2
}

# 輸出結果
print(f"📌 搜尋條件：search_keyword 含『{search_keyword}』")
if filter_keywords_in_title:
    print(f"🎯 進階條件：title 同時包含『{'、'.join(filter_keywords_in_title)}』\n")

if not filtered_grouped:
    print("⚠️ 查無重複價格變化的項目")
else:
    for title, records in filtered_grouped.items():
        print(f"🔹 標題：{title}")
        for price, timestamp, source in sorted(records, key=lambda x: x[1]):
            print(f"   - 價格：{price}（時間：{timestamp}，來源：{source}）")
        print("-" * 60)

📌 搜尋條件：search_keyword 含『BLUE BAY 倍力』
🎯 進階條件：title 同時包含『室內低磷』

🔹 標題：BLUE BAY 倍力 Animate 室內低磷配方 乾飼料, 海魚 + 鮮雞, 4.5kg, 1袋
   - 價格：$791（時間：2025-05-18 02:51:09，來源：login）
   - 價格：$791（時間：2025-05-18 15:34:38，來源：login）
   - 價格：$795（時間：2025-05-19 03:27:47，來源：login）
   - 價格：$795（時間：2025-05-19 04:08:37，來源：login）
   - 價格：$795（時間：2025-05-19 04:10:15，來源：login）
------------------------------------------------------------
